# Classification

- Dataset: Bank Marketing Data Set https://archive.ics.uci.edu/ml/datasets/bank+marketing

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


DF_RAW = pd.read_csv(
    "bank-full.csv",
    sep=';'
)

In [2]:
# Confirm dataset content
display(DF_RAW.head())

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
print(f"shape:{DF_RAW.shape}\n")
print(f"y:\n{DF_RAW['y'].value_counts()}\n")
print(
    f"Sales success rate = {DF_RAW['y'].value_counts()['yes']/len(DF_RAW)}"
)

shape:(45211, 17)

y:
no     39922
yes     5289
Name: y, dtype: int64

Sales success rate = 0.11698480458295547


In [4]:
print(DF_RAW.isnull().sum())

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64
